In [6]:
import sys
sys.path.append('..')
from Comms_System import Comms_System, SNR_plot, SNR_plot_vanilla
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.axes as axes
#import scipy.stats
#from ML_components import load_params
#from Network import NeuralNetwork
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()

In [ ]:
print('Yo, welcome to Main CS')

## Automatic Tests

In [32]:
#symbol_set = [7, 5, 3, 1, -1, -3, -5, -7] # all symbols that we use
symbol_set = [3, 1, -1, 3]
num_symbols = 10000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, norm_h=True)
SNR = 10
sigma = CS.SNRdb_to_sigma(SNR, use_gain=True)
print(sigma)

#sigma = 0.35 # corresponds roughly to SNR 16 (old sigma=1)
#sigma = 0.7 # corresponds roughly to SNR 10 (old sigma=2)
#sigma = 1.06 # corresponds roughly to SNR 6.4 (old sigma=3)

decisions = CS.transmission(noise_level=sigma, mode='euclidean', norm_signal=False)
CS.evaluate(decisions)

0.7095348738859334
E: 0.9999994781112497
Ratio: 10.0


(963, 0.0963)

In [ ]:
SNRdbs, euclid_error_rates, error_theory = SNR_plot_vanilla(norm_h=False, use_gain=True)
# can be True, True
# True, False
# False, True
# but not False, False

### Manual Test

In [ ]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 1000

symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=0.35, norm_h=False)
symbol_seq_power = np.mean(np.square(symbol_seq))
print("Symbol seq power:", symbol_seq_power)
sigma = 1

# calibrate
gain_factor = np.max(np.convolve(CS.h, CS.h))
gain_tx = np.max(CS.h)
print("gain_factor:", gain_factor)

# upsample symbol sequence and filter it on transmission side
upsampled = CS.upsample(v=False)
Tx = np.convolve(upsampled, CS.h)

#plt.title('Before entering channel')
#plt.plot(Tx)
#plt.show()

# Normalize signal with RMS
Tx = Tx / np.sqrt(np.mean(np.square(Tx)))
print("power of normalized signal:", np.mean(np.square(Tx)))
#Tx = Tx / np.max(Tx)


# Transmit the filtered signal (i.e. add noise)
Tx = Tx + np.random.normal(0.0, 0, Tx.shape)  # add gaussian noise

# Filter on receiver side
Rx = np.convolve(Tx, CS.h)#/np.sqrt(gain_factor)
print("Power of Rx:", np.mean(np.square(Rx)))

# Normalize received signal with RMS * squareroot of symbol seq power?
Rx = (Rx / np.sqrt(np.mean(np.square(Rx)))) * np.sqrt(symbol_seq_power)
print("Power of Normalized Rx:", np.mean(np.square(Rx)))
#plt.title('After Filtering on Receiver')
#plt.plot(Rx)
#plt.show()

# Downsample the signal on the receiver side
downsampled = CS.downsample(Rx) #/gain_factor
print("power of downsampled:", np.mean(np.square(downsampled)))

# Decision-making using new_values
decisions = CS.decision_making(downsampled, False)

CS.evaluate(decisions)

## Display filter

In [ ]:
betas = [0, 0.35, 0.7, 1]
plt.figure(figsize=(13,8))
plt.title('Root Raised Cosine', fontsize=24)
plt.xlabel('Time', fontsize=20)
plt.ylabel('Amplitude', fontsize=20)
z = np.zeros(len(CS.h))
for beta in betas:
    CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=beta, norm_h=True)
    #f = np.fft.fft(CS.h)
    plt.plot(CS.h)
plt.grid(True)
plt.legend(betas, fontsize=12)
plt.show()


plt.figure(figsize=(13,8))
plt.title('Raised Cosine', fontsize=24)
plt.xlabel('Time', fontsize=20)
plt.ylabel('Amplitude', fontsize=20)
z = np.zeros(len(CS.h))
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=0.35, norm_h=True)
plt.stem(np.convolve(CS.h, CS.h))
plt.grid(True)
plt.show()

____

# Evaluation of pretrained Decision Making models

In [ ]:
for i in range(5):
    symbol_set = [3, 1, -1, -3] # all symbols that we use
    num_symbols = 10000
    symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
    m = 8
    CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, norm_h=True)
    noise_level = CS.SNR_to_sigma(16)
    
    # Automatic test
    euclid_decisions, NN_decisions, block_decisions, filter_decisions, conv_decisions, _ = CS.test_CS(noise_level=noise_level)
    print('Euclidean Distance Error Rate: {}%'.format((CS.evaluate(euclid_decisions)[1]*100).round(2)))
    #print('NN Decision Making Error Rate: {}%'.format((CS.evaluate(NN_decisions)[1]*100).round(2)))
    #print('NN Block Error Rate: {}%'.format((CS.evaluate(block_decisions)[1]*100).round(2)))
    #print('NN Filter Error Rate: {}%'.format((CS.evaluate(filter_decisions)[1]*100).round(2)))
    print('Conv Filter Error Rate: {}%'.format((CS.evaluate(conv_decisions)[1]*100).round(2)))

    print()

### SNR Calculations

In [ ]:
SNRsDB, euclid_er, NN_er, NN_blocks_er, NN_filter_er, conv_er, error_theory = SNR_plot(100000, lowpass=None, 
                                                                                       conv_model=None, 
                                                                                       norm_h=True)

In [ ]:
plt.figure(figsize=(18,11))
plt.title('Noise Plot with lowpass', fontsize=24)
plt.xlabel('SNR (dB)', fontsize=20)
plt.ylabel('$P_e$', fontsize=20)
num = 0
plt.semilogy(SNRsDB[num:], euclid_er[num:], alpha=1)
#plt.semilogy(SNRsDB[num:], NN_er[num:], alpha=1)
#plt.semilogy(SNRsDB[num:], NN_blocks_er[num:])
#plt.semilogy(SNRsDB[num:], NN_filter_er[num:])
#plt.semilogy(SNRsDB[num:], conv_er[num:])
plt.semilogy(SNRsDB[num:], error_theory[num:])
plt.legend(['Euclidean', 'Theory'], fontsize=16)
#plt.ylim([1e-3, 1])
plt.show()

In [ ]:
error_rate_for_sigma3 = 1.5 * (1 - scipy.stats.norm.cdf(np.sqrt(gain_factor/(3**2))))
error_rate_for_sigma2 = 1.5 * (1 - scipy.stats.norm.cdf(np.sqrt(gain_factor/(2**2))))
error_rate_for_sigma1 = 1.5 * (1 - scipy.stats.norm.cdf(np.sqrt(gain_factor/(1**2))))

# Inspect Decision Boundaries

In [ ]:
weights, biases, sizes = load_params('../Weights/decision_making_weights.npz',
                                         '../Weights/decision_making_biases.npz')
NN = NeuralNetwork(sizes, startweights=weights, startbiases=biases,
                           type='classification', afunc='relu')
classes = np.array(symbol_set)

## Plot 1

In [ ]:
test_seq = np.linspace(-4, 4, 40)
test_seq = np.array(test_seq, ndmin=2).T
yhat_onehot = NN.feedforward(test_seq)


plt.figure(figsize=(13,8))
plt.title('Decision Boundaries', fontsize=24)
for i in range(yhat_onehot.shape[1]):
    plt.plot(test_seq, yhat_onehot[:,i], '-x')
#plt.axvline(x=-2, color='black')
#plt.axvline(x=0, color='black')
#plt.axvline(x=2, color='black')
plt.xlabel('Input', fontsize=20)
plt.ylabel('Probability of choosing', fontsize=20)
plt.legend(symbol_set + ['Euclidean Boundaries'], fontsize=14)
plt.show()

## Plot 2

In [ ]:
classes = np.array(symbol_set)
test_seq = np.linspace(-4, 4+1, 100)
test_seq = np.array(test_seq, ndmin=2).T

yhat_idx = NN.feedforward(test_seq).argmax(axis=1)
yhat = classes[yhat_idx]
euclid_decisions = CS.decision_making(test_seq)


num = len(test_seq)
plt.figure(figsize=(13,8))
plt.xlabel('Sample Value (input)', fontsize=20)
plt.ylabel('Decided value (output)', fontsize=20)
plt.plot(test_seq[:num], yhat[:num], '-rx', alpha=0.5)
plt.plot(test_seq[:num], euclid_decisions[:num], '-bx', alpha=1)
plt.legend(['Neural Network', 'Euclidean'], fontsize=14)
plt.show()

##### Want to time something? Use following code stump

In [ ]:
# %%timeit -o -r 5